# Decorator

简单地说：decorator是修改其他函数的功能的函数，有助于让代码更简短，更Pythonic。

## 一切皆对象

函数是一等公民，可以将函数赋给一个变量。新的变量对函数建立了引用(指向)。

In [ ]:
def hi(name="yasoob"):
    return "hi" + name

print(hi())  # outputs: 'hi yasoob'

greet = hi  # 这里没有使用小括号，因为并不是在调用hi函数
print(greet())

del hi  # 如果删掉hi函数，会发生什么

print(hi())  # outputs: NameError
print(greet())  # outputs: 'hi yasoob'


## 在函数中可以定义函数

In [ ]:
def hi(name="yasoob"):
    print("now you are inside the hi() function")
    
    def greet():
        return "now you are in the greet() function"
    
    def welcome():
        return "now you are in the welcome() function"
    
    print(greet())
    print(welcome())
    print("now you are back in the hi() function")
    

上面的代码展示了无论何时调用hi(),greet()和welcome()将会同时被调用。greet()和welcome()函数在hi()函数外是不能访问的，如直接调用：

greet()  # outputs: NameError: name 'greet' is not defined

## 从函数中返回函数

其实并不需要在一个函数里去执行另一个函数，也可以将其作为输出返回出来：

In [ ]:
def hi(name="yasoob"):
    def greet():
        return "now you are in the greet() function"
    
    def welcome():
        return "now you are in the welcome() function"
    
    if name == "yasoob":
        return greet
    else:
        return welcome
    
a = hi()
print(a)
# outputs: <function greet at 0x7f2143c01500>
# 上面的代码展示了`a`现在指向到hi()函数中的greet()函数

print(a())
# outputs: now you are in the greet() function

# 注意：在if/else语句中我们返回greet和welcome，而不是greet()和welcome().为什么是那样？因为当你把小括号放在函数名后面，这个函数就会执行；
# 然而，如果你不放括号在它后面，那它可以被到处传递，并且可以复制给别的变量而不去执行它。
    

## 将函数作为参数传给另一个函数

In [ ]:
def hi():
    return "hi yasoob!"

def doSomethingBeforeHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())
    
    
doSomethingBeforeHi(hi)

## 第一个装饰器

In [ ]:
def a_new_decorator(a_func):
    
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
        
    return wrapTheFunction


def a_func_requiring_decoration():
    print("I am the function which needs some decoration to remove my work")
    
a_func_requiring_decoration = a_new_decorator(a_func_requiring_decoration)
a_func_requiring_decoration()

In [ ]:
from functools import wraps

def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated

@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())
can_run = False
print(func())

## 使用场景

    - 授权
    - 日志

### 1 授权Authorization

装饰器能有助于检查某个人是否被授权去使用一个web应用的端点(endpoint)。他们被大量用于Flask和Django Web框架中。下面是一个例子来使用基于
装饰器的授权：

In [ ]:
from functools import wraps

def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authoriztion
        if not auth or not chech_auth(auth.username, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated


### 2 日志Logging

日志是装饰器运用的另一个亮点：

In [ ]:
from functools import wraps

def logit(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + "was called")
        return func(*args, **kwargs)
    return logit

@logit
def addition_func(x):
    """Do some math."""
    return x + x

result = addition_func(4)

## 带参数的装饰器

### 在函数中嵌入装饰

我们回到日志的例子，并创建一个包裹函数，能让我们指定一个用于输出的日志文件。

In [ ]:
from functools import wraps

def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + "was called"
            print(log_string)
            # 打开logfile，并写入内容
            with open(logfile, 'a') as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator

@logit()
def myfunc1():
    pass

myfunc1()
# Output: myfunc1 was called
# 现在一个叫做 out.log 的文件出现了,里面的内容就是上面的字符串

@logit(logfile='func2.log')
def myfunc2():
    pass

myfunc2()
# Output: myfunc2 was called
# 现在一个叫做 func2.log 的文件出现了,里面的内容就是上面的字符串


### 装饰器类

现在我们有了能用于正式环境的logit装饰器，但当我们的应用的某些部分还比较脆弱时，异常也许是需要更紧急关注的事情。比方说有时你指想打日志到
一个文件。而有事你向把引起你注意的问题发送到一个email，同时也保留日志，留个记录。这是一个使用继承的场景，但目前为止我们只看到过用来构建
装饰器的函数。
幸运的是，类也可以用来构建装饰器。那我们现在以一个类而不是一个函数的方式，来重新构建logit.

In [ ]:
from functools import wraps

class logit(object):
    def __init__(self, logfile='out.log'):
        self.logfile = logfile
        
    def __call__(self, func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + "was called"
            print(log_string)
            # 打开logfile并写入
            with open(self.logfile, 'a') as opened_file:
                # 现在将日志打到指定的文件
                opened_file.write(log_string + '\n')
            # 现在，发送一个通知
            self.notify()
            return func(*args, **kwargs)
        return wrapped_function
    
    def notify(self):
        # logit只打日志，不做别的
        pass

# 这个实现有一个附加优势，在于比嵌套函数的方式更加整洁，而且包裹一个函数还是使用跟以前一样的语法：

@logit()
def myfunc1():
    pass

# 现在，我们给logit创建子类，来添加email的功能(虽然email这个话题不会在这里展开)。

class email_logit(logit):
    '''
    一个logit的实现版本，可以在函数调用时发送email给管理员
    '''
    def __init__(self, email='admin@myproject.com', *args, **kwargs):
        self.email = email
        super(email_logit, self).__init__(*args, **kwargs)
        
    def notify(self):
        # 发送一封email到self.email
        # 这里就不坐实现了
        pass
    
# 从现在其，@email_logit将会和@logit产生同样的效果，但是在打日志的基础上，还会多发送一封邮件给管理员。